In [ ]:
import tensorflow as tf
tf.random.set_seed(1234)
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import pickle
import math

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/

/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

170498071/170498071 [==============================] - 6s 0us/step


In [ ]:
train_labels1 = tf.keras.utils.to_categorical(train_labels)
test_labels1 = tf.keras.utils.to_categorical(test_labels)

In [ ]:
BATCH_SIZE=64
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels1))
train_ds = train_ds.shuffle(train_images.shape[0]).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels1))
test_ds = train_ds.shuffle(test_images.shape[0]).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
class CustomMLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred))
        #element_wise = tf.math.add(x=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred),
        #                   y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0, y=y_true), y=tf.math.subtract(x=1.0, y=y_pred)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class ParamMLoss(tf.keras.losses.Loss):
    def __init__(self,alpha):
        self.alpha=alpha
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(y_true, tf.math.subtract(tf.math.divide_no_nan(1.0, tf.math.pow(y_pred,self.alpha)), 1.0))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.divide_no_nan(x=y_true,
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0,
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             ))

        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLFullLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.subtract(x=tf.math.add(x=tf.math.divide_no_nan(x=y_true,
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0,
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             )),
                                                      y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0,y=y_true),
                                                                              y=tf.math.sqrt(
                                                                                  tf.math.subtract(x=1.0,y=tf.math.square(y_pred))
                                                                              ))),
                                        y=1.0)

        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class My_CE_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        log_y_pred = tf.math.log(y_pred)
        element_wise = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class ParabolicLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.subtract(x=1.0, y=tf.math.square(y_pred)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CircleLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.sqrt(tf.math.subtract(x=1.0, y=tf.math.square(y_pred))))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CosLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(y_true, tf.math.cos(tf.math.divide_no_nan(tf.math.multiply_no_nan(math.pi, y_pred), 2.0)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import SGD, Adam

base_model = VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(32, 32, 3),
    classes=10
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='tanh')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

opt = Adam(learning_rate=1e-5)
model.compile(optimizer=opt, loss=CustomMLoss(), metrics=['accuracy'])

80134624/80134624 [==============================] - 4s 0us/step


In [ ]:
if 'COLAB_TPU_ADDR' in os.environ:
  TPU_WORKER = 'grpc://{0}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TPU_WORKER=''
print(TPU_WORKER)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print(tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

grpc://10.50.202.186:8470


[LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [ ]:
def create_model():
    inputs = tf.keras.layers.Input(shape=(32,32,3))
    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    base_model_output = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        input_shape=(224, 224, 3),
        weights='imagenet'
    )(resize)

    x = tf.keras.layers.GlobalAveragePooling2D()(base_model_output)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    predictions = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
with strategy.scope():
    model = create_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=ParamMLoss(alpha=1.0),
        metrics=['accuracy'],
    )

In [ ]:
model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=ParamMLoss(alpha=1.0),
    metrics=['accuracy'],
)
history = model.fit(train_ds, epochs=10, verbose=2)

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
782/782 - 741s - loss: 4.0486 - accuracy: 0.5690 - 741s/epoch - 947ms/step
Epoch 2/10
782/782 - 730s - loss: 2.1524 - accuracy: 0.7542 - 730s/epoch - 933ms/step
Epoch 3/10
782/782 - 727s - loss: 1.4945 - accuracy: 0.8153 - 727s/epoch - 930ms/step
Epoch 4/10
782/782 - 728s - loss: 1.3022 - accuracy: 0.8370 - 728s/epoch - 931ms/step
Epoch 5/10
782/782 - 729s - loss: 1.0427 - accuracy: 0.8635 - 729s/epoch - 932ms/step
Epoch 6/10
782/782 - 730s - loss: 0.8339 - accuracy: 0.8829 - 730s/epoch - 934ms/step
Epoch 7/10
782/782 - 729s - loss: 1.1670 - accuracy: 0.8553 - 729s/epoch - 932ms/step
Epoch 8/10
782/782 - 724s - loss: 0.6741 - accuracy: 0.9000 - 724s/epoch - 926ms/step
Epoch 9/10


KeyboardInterrupt: ignored

In [ ]:
train_labels1 = tf.keras.utils.to_categorical(train_labels)
test_labels1 = tf.keras.utils.to_categorical(test_labels)

train_labels1 = tf.convert_to_tensor(train_labels1, dtype=tf.float32)
test_labels1 = tf.convert_to_tensor(test_labels1, dtype=tf.float32)

history = model.fit(train_images, train_labels1, epochs=10, batch_size=64,
                    verbose=2, validation_split=0.2)

Epoch 1/10
625/625 - 661s - loss: 0.9025 - accuracy: 0.1005 - val_loss: 0.9023 - val_accuracy: 0.0977 - 661s/epoch - 1s/step
Epoch 2/10


KeyboardInterrupt: ignored

In [ ]:
#M
train_loss, train_acc = model.evaluate(train_images,  train_labels1, verbose=0)
test_loss, test_acc = model.evaluate(test_images,  test_labels1, verbose=0)
print(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')
print(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')

The training accuracy was 0.5538 and validation accuracy was 0.5489 
The training loss was 4.2945 and validation loss was 4.4987 


In [ ]:
hist = history.history
with open('/content/gdrive/MyDrive/Colab Notebooks/m_cifar_hist.pickle', 'wb') as handle:
    pickle.dump(hist, handle)

In [ ]:
#M Full
train_loss, train_acc = model.evaluate(train_images,  train_labels1, verbose=0)
test_loss, test_acc = model.evaluate(test_images,  test_labels1, verbose=0)
print(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')
print(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')

The training accuracy was 0.555 and validation accuracy was 0.549 
The training loss was 5.1558 and validation loss was 5.3624 


In [ ]:
hist = history.history
with open('/content/gdrive/MyDrive/Colab Notebooks/m_full_cifar_hist.pickle', 'wb') as handle:
    pickle.dump(hist, handle)

In [ ]:
#L Full
train_loss, train_acc = model.evaluate(train_images,  train_labels1, verbose=0)
test_loss, test_acc = model.evaluate(test_images,  test_labels1, verbose=0)
print(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')
print(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')

The training accuracy was 0.5594 and validation accuracy was 0.555 
The training loss was 0.6827 and validation loss was 0.7018 


In [ ]:
hist = history.history
with open('/content/gdrive/MyDrive/Colab Notebooks/l_full_cifar_hist.pickle', 'wb') as handle:
    pickle.dump(hist, handle)

In [ ]:
#L
train_loss, train_acc = model.evaluate(train_images,  train_labels1, verbose=0)
test_loss, test_acc = model.evaluate(test_images,  test_labels1, verbose=0)
print(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')
print(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')

The training accuracy was 0.5561 and validation accuracy was 0.55 
The training loss was 1.6208 and validation loss was 1.6409 


In [ ]:
hist = history.history
with open('/content/gdrive/MyDrive/Colab Notebooks/l_cifar_hist.pickle', 'wb') as handle:
    pickle.dump(hist, handle)

In [ ]:
#CE
train_loss, train_acc = model.evaluate(train_images,  train_labels1, verbose=0)
test_loss, test_acc = model.evaluate(test_images,  test_labels1, verbose=0)
print(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')
print(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')

The training accuracy was 0.5529 and validation accuracy was 0.545 
The training loss was 1.2969 and validation loss was 1.3162 


In [ ]:
hist = history.history
with open('/content/gdrive/MyDrive/Colab Notebooks/ce_cifar_hist.pickle', 'wb') as handle:
    pickle.dump(hist, handle)